# Imports 

## Packages 

In [1]:
# Ne pas oublier d'executer dans le shell avant de lancer python :
# source /users/Enseignants/piwowarski/venv/amal/3.7/bin/activate
# N.B: commande pour lancer tensorboard: tensorboard --logdir=path/to/log-directory

import torch
from torch.autograd import Function
from torch.autograd import gradcheck
from datamaestro import prepare_dataset 
from torch.utils.tensorboard import SummaryWriter
from torch.optim import adadelta
import ipdb
import matplotlib.pyplot as plt

## TP1 classes 

In [2]:
class Context:
    """Very simplified context object"""
    def __init__(self):
        self._saved_tensors = ()
    def save_for_backward(self, *args):
        self._saved_tensors = args
    @property
    def saved_tensors(self):
        return self._saved_tensors


class linear(Function):
##Toute fonction a:
## une méthode forward pour calculer l'image de variables et paramètres donnés
## une méthode backward pour renvoyer son gradient par rapport à ses variables/paramètres
    @staticmethod
    #a static method can be called without referring to an object.     
    def forward(ctx,x,w,b):
        ctx.save_for_backward(x,w,b)
        return w*x+b

    @staticmethod
    def backward(ctx,grad_output):
        ## prend comme argument le gradient de l'étage d'au dessus
        x,w,b = ctx.saved_tensors
        dL_dx=grad_output*w
        dL_dw=grad_output*x
        dL_db=grad_output
        return dL_dx,dL_dw,dL_db
    
class MSE(Function):
    @staticmethod
    def forward(ctx,y,yhat):
        ctx.save_for_backward(y,yhat)
        return torch.mean((y-yhat)**2)
    @staticmethod
    def backward(ctx,grad_output=1):
        y,yhat=ctx.saved_tensors
        return 2*(y-yhat)*grad_output


## Data import 

In [3]:
ds=prepare_dataset("edu.uci.boston")
fields, data =ds.files.data()
n = data.shape[0]

def standardize(z):
    m=z.mean(dim=0,keepdim=True)
    s=z.std(dim=0,keepdim=True)
    return (z-m)/s

x=standardize(torch.tensor(data[:,:-1],dtype=torch.float32))
y=standardize(torch.tensor(data[:,-1],dtype=torch.float32))

# Différentiation automatique 

In [4]:
def linReg(x,y,learning_rate=0.001,epochs=100):

    w=torch.randn(x.shape[1],requires_grad=True,dtype=x.dtype)
    b=torch.randn(1,requires_grad=True,dtype=x.dtype)
    writer=SummaryWriter()
    for i in range(epochs):
        for j in range(len(x)):
            w.requires_grad=True
            b.requires_grad=True
            x_j=x[j,:]
            y_j=y[j]
#             ipdb.set_trace()
            y_hat=torch.dot(w,x_j)+b
            loss=(y_j-y_hat)**2
            loss.backward()
            ipdb.set_trace()
            w=w-learning_rate*w.grad
            #met à jour w,b en fonction d'eux même (impossible?)             
            b=b-learning_rate*b.grad
            w.grad.data.zero_()
            b.grad.data.zero_()
        writer.add_scalar('Loss LinReg',loss,i)
    return w,b

In [5]:

# w,b=linReg(x,y)

## Optimiseur 

#### SGD

In [6]:
learning_rate=1e-2
nb_epochs=1000

def f(x,w,b):
    return torch.mv(x,w)+b

def MSE(y,y_hat):
    return torch.mean(torch.pow((y-y_hat),2))


w=torch.nn.Parameter(torch.randn(x.shape[1]))
b=torch.nn.Parameter(torch.randn(1))

optim=torch.optim.SGD(params=[w,b],lr=learning_rate)
# Configuration de l'optimiseur: paramètres et critère d'arrêt
optim.zero_grad()
# Réinitialisation du gradient

writer=SummaryWriter()
for i in range(nb_epochs):
    index=torch.randint(0,len(x),size=(1,))
    #Forward     
    loss=MSE(f(x[index,:],w,b),y[index])
    writer.add_scalar("SGD loss",loss,i)
    #loss_history.append(loss)
    #Backward
    loss.backward()
    #Mise à jour des paramètres à chaque epoch (mod 1)
    if i % 1==0:
        optim.step() #met à jour les paramètres [w,b]
        optim.zero_grad() #réinitialise le gradient
    if i%100==0:
        print("Epoch",i)
        
# plt.plot(loss_history)
# plt.show()

Epoch 0
Epoch 100
Epoch 200
Epoch 300
Epoch 400
Epoch 500
Epoch 600
Epoch 700
Epoch 800
Epoch 900


#### Batch

In [7]:
nb_epochs=1000
learning_rate=1e-3

def f(x,w,b):
#     ipdb.set_trace()
    return torch.mv(x,w)+b

def MSE(y,y_hat):
    return torch.mean(torch.pow((y-y_hat),2))

w=torch.nn.Parameter(torch.randn(x.shape[1]))
b=torch.nn.Parameter(torch.randn(1))

optim=torch.optim.SGD(params=[w,b],lr=learning_rate)
# Configuration de l'optimiseur
optim.zero_grad()
# Réinitialisation du gradient

writer=SummaryWriter()
for i in range(nb_epochs):
#     for j in range(len(x)):
    #Forward     
    loss=MSE(f(x,w,b),y)
    #Backward
    loss.backward()
    writer.add_scalar("Batch loss",loss,i)
    if i % 1==0:
        optim.step() #met à jour les paramètres [w,b]
        optim.zero_grad() #réinitialise le gradient
    if i%100==0:
        print("Epoch ",i)


Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900


#### Mini-Batch 

In [8]:
nb_epochs=1000
batch_size=10

def f(x,w,b):
#     ipdb.set_trace()
    return torch.mv(x,w)+b

def MSE(y,y_hat):
    return torch.mean(torch.pow((y-y_hat),2))

w=torch.nn.Parameter(torch.randn(x.shape[1]))
b=torch.nn.Parameter(torch.randn(1))

optim=adadelta.Adadelta(params=[w,b])
# Configuration de l'optimiseur
optim.zero_grad()
# Réinitialisation du gradient

writer=SummaryWriter()
for i in range(nb_epochs):
    index=torch.randint(0,len(x),size=(batch_size,))
    #Forward     
    loss=MSE(f(x[index,:],w,b),y[index])
    #Backward
    loss.backward()
    writer.add_scalar("MiniBatch loss",loss,i)
    if i % 1==0:
        optim.step() #met à jour les paramètres [w,b]
        optim.zero_grad() #réinitialise le gradient
    if i%100==0:
        print("Epoch ",i)


Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900


## Module 

### 2-layers NN (without container) (SGD algorithm method)

In [9]:
nb_epochs=1000
learning_rate=1e-3

f1=torch.nn.Linear(x.shape[1],1)
tanh=torch.nn.Tanh()
f2=torch.nn.Linear(1,1)
mse=torch.nn.MSELoss()

# Set optimizer
optim=torch.optim.SGD(params=[f1.weight,f1.bias,f2.weight,f2.bias],lr=learning_rate)
optim.zero_grad()

writer=SummaryWriter()
for i in range(nb_epochs):
    index=torch.randint(0,len(x),size=(1,))
    y1=f1(x[index,:])
    z=tanh(y1)
    y2=f2(z)
    loss=mse(y[index],y2)
    loss.backward()
    writer.add_scalar("SGD loss",loss.item(),i)
    optim.step()
    optim.zero_grad()
    if i %100==0:
        print("Epoch ",i)
    


/home/paul/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900


### Container (Mini batch used)

In [10]:
nb_epochs=1000
batch_size=10
learning_rate=1e-2

f1=torch.nn.Linear(x.shape[1],1)
tanh=torch.nn.Tanh()
f2=torch.nn.Linear(1,1)
mse=torch.nn.MSELoss()

neural_network=torch.nn.Sequential(f1,tanh,f2)

optim=torch.optim.SGD(params=list(neural_network.parameters()),lr=learning_rate)
for i in range(nb_epochs):
    index=torch.randint(0,len(x),size=(batch_size,))
    loss=mse(neural_network(x[index,:]),y[index].reshape(len(index),1))
    loss.backward()
    writer.add_scalar("MiniBatch loss",loss.item(),i)
    if i%100==0:
        print("Epoch ",i)
    

Epoch  0
Epoch  100
Epoch  200
Epoch  300
Epoch  400
Epoch  500
Epoch  600
Epoch  700
Epoch  800
Epoch  900
